In [1]:
#!/usr/bin/env python3

import os
import pandas as pd
import pickle

aqm_config = ["fifo", "fq_codel", "red", "red_noecn"]

speed_config = [
    {"tag": "100mbps", "speed": 100000000, "processes": 1, "parallel_streams": 1},
    {"tag": "500mbps", "speed": 500000000, "processes": 5, "parallel_streams": 1},
    {"tag": "1gbps", "speed": 1000000000, "processes": 10, "parallel_streams": 1},
    {"tag": "10gbps", "speed": 10000000000, "processes": 20, "parallel_streams": 5},
    {"tag": "25gbps", "speed": 25000000000, "processes": 25, "parallel_streams": 10}
]

cca_config = [
    {"cca1": "bbr", "cca2": "cubic"},
    {"cca1": "bbr2", "cca2": "cubic"},
    {"cca1": "htcp", "cca2": "cubic"},
    {"cca1": "reno", "cca2": "cubic"},
    {"cca1": "cubic", "cca2": "cubic"},
    {"cca1": "bbr", "cca2": "bbr"},
    {"cca1": "bbr2", "cca2": "bbr2"},
    {"cca1": "htcp", "cca2": "htcp"},
    {"cca1": "reno", "cca2": "reno"}
]

bdp_config = [0.5, 1, 2, 4, 8, 16]

sender_1_output = "poseidon-sender-1/output"
sender_2_output = "poseidon-sender-2/output"

rows = []
for aqm in aqm_config:
    for cca_combo in cca_config:
        for speed in speed_config:
            for bdp in bdp_config:
                for run in range(1,6):
                    row = {
                        "aqm": aqm, 
                        "cca1": cca_combo['cca1'],
                        "cca2": cca_combo['cca2'],
                        "speed": speed['tag'], 
                        "bdp": bdp,
                        "run": run
                    }

                    #sender 1
                    f1 = f"raw_data/{sender_1_output}/{speed['tag']}_{cca_combo['cca1']}_{cca_combo['cca2']}_{aqm}_{bdp}bdp_{run}.total"
                    #semder 2
                    f2 = f"raw_data/{sender_2_output}/{speed['tag']}_{cca_combo['cca2']}_{cca_combo['cca1']}_{aqm}_{bdp}bdp_{run}.total"
                    
                    with open(f1, 'r') as f:
                        lines = f.readlines()
                        measurements = list(map(float, lines[1].replace("\n",'').split(",")))
                        row["sender_1_goodput"] = measurements[0]
                        row["sender_1_retransmits"] = measurements[1]

                    with open(f2, 'r') as f:
                        lines = f.readlines()
                        measurements = list(map(float, lines[1].replace("\n",'').split(",")))
                        row["sender_2_goodput"] = measurements[0]
                        row["sender_2_retransmits"] = measurements[1]
                    
                    rows.append(row)

results = pd.DataFrame.from_dict(rows)

In [2]:
#with open('results.pickle', 'wb') as handle:
#    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

agg_results = results.drop(columns=["run"]).groupby(by=["aqm", "cca1", "cca2", "speed", "bdp"]).aggregate("mean")

In [3]:
agg_results["fairness_index"] = round(((agg_results["sender_1_goodput"]+agg_results["sender_2_goodput"])**2)/(2.0*(agg_results["sender_1_goodput"]**2 + agg_results["sender_2_goodput"]**2)), 4)
agg_results["total_mean_goodput"] = agg_results["sender_1_goodput"] + agg_results["sender_2_goodput"]
agg_results["total_mean_retx_packets"] = agg_results["sender_1_retransmits"] + agg_results["sender_2_retransmits"]

In [4]:
only_cubic = results[(results["cca1"] == "cubic") & (results["cca2"] == "cubic")]
only_cubic = only_cubic.drop(columns=["run", "cca1", "cca2"]).groupby(by=["aqm", "speed", "bdp"]).aggregate("mean")

In [5]:
only_cubic["fairness_index"] = round(((only_cubic["sender_1_goodput"]+only_cubic["sender_2_goodput"])**2)/(2.0*(only_cubic["sender_1_goodput"]**2 + only_cubic["sender_2_goodput"]**2)), 4)
only_cubic["total_mean_goodput"] = only_cubic["sender_1_goodput"] + only_cubic["sender_2_goodput"]
only_cubic["total_mean_retx_packets"] = only_cubic["sender_1_retransmits"] + only_cubic["sender_2_retransmits"]

In [6]:
only_cubic = only_cubic.drop(columns=["sender_1_goodput", "sender_1_retransmits", "sender_2_goodput", "sender_2_retransmits"]).reset_index()
agg_results = agg_results.drop(columns=["sender_1_goodput", "sender_1_retransmits", "sender_2_goodput", "sender_2_retransmits"]).reset_index()

In [7]:
only_cubic

,aqm,speed,bdp,fairness_index,total_mean_goodput,total_mean_retx_packets
0,fifo,100mbps,0.5,0.9997,100.78,2587.0
1,fifo,100mbps,1.0,0.9942,100.84,2398.6
2,fifo,100mbps,2.0,0.9985,100.94,2354.6
3,fifo,100mbps,4.0,0.9973,102.20,2408.6
4,fifo,100mbps,8.0,0.9884,101.56,2790.0
...,...,...,...,...,...,...
115,red_noecn,500mbps,1.0,1.0000,466.90,5325.4
116,red_noecn,500mbps,2.0,1.0000,467.78,5283.6
117,red_noecn,500mbps,4.0,0.9999,467.34,5319.8
118,red_noecn,500mbps,8.0,0.9997,467.66,5286.0


In [8]:
agg_results

,aqm,cca1,cca2,speed,bdp,fairness_index,total_mean_goodput,total_mean_retx_packets
0,fifo,bbr,bbr,100mbps,0.5,0.9993,101.64,38771.8
1,fifo,bbr,bbr,100mbps,1.0,0.8975,101.62,40820.6
2,fifo,bbr,bbr,100mbps,2.0,0.9996,101.66,15952.6
3,fifo,bbr,bbr,100mbps,4.0,1.0000,102.08,4192.2
4,fifo,bbr,bbr,100mbps,8.0,0.9999,102.36,4839.4
...,...,...,...,...,...,...,...,...
1075,red_noecn,reno,reno,500mbps,1.0,1.0000,414.76,7854.0
1076,red_noecn,reno,reno,500mbps,2.0,1.0000,414.94,7915.4
1077,red_noecn,reno,reno,500mbps,4.0,1.0000,415.02,7941.4
1078,red_noecn,reno,reno,500mbps,8.0,1.0000,415.26,7891.0


In [9]:
agg_results_join = pd.merge(agg_results, only_cubic, how="left", on=["aqm", "speed", "bdp"], suffixes=("","_cubic"))

In [10]:
speed_config_df = pd.DataFrame.from_dict(speed_config).drop(columns=["processes", "parallel_streams"])
speed_config_df = speed_config_df.rename(columns={"speed": "speed_bits"})
agg_results_join = pd.merge(agg_results_join, speed_config_df, how="left", left_on=["speed"], right_on=["tag"], suffixes=("","_cubic")).drop(columns=["tag"])

In [11]:
agg_results_join

,aqm,cca1,cca2,speed,bdp,fairness_index,total_mean_goodput,total_mean_retx_packets,fairness_index_cubic,total_mean_goodput_cubic,total_mean_retx_packets_cubic,speed_bits
0,fifo,bbr,bbr,100mbps,0.5,0.9993,101.64,38771.8,0.9997,100.78,2587.0,100000000
1,fifo,bbr,bbr,100mbps,1.0,0.8975,101.62,40820.6,0.9942,100.84,2398.6,100000000
2,fifo,bbr,bbr,100mbps,2.0,0.9996,101.66,15952.6,0.9985,100.94,2354.6,100000000
3,fifo,bbr,bbr,100mbps,4.0,1.0000,102.08,4192.2,0.9973,102.20,2408.6,100000000
4,fifo,bbr,bbr,100mbps,8.0,0.9999,102.36,4839.4,0.9884,101.56,2790.0,100000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1075,red_noecn,reno,reno,500mbps,1.0,1.0000,414.76,7854.0,1.0000,466.90,5325.4,500000000
1076,red_noecn,reno,reno,500mbps,2.0,1.0000,414.94,7915.4,1.0000,467.78,5283.6,500000000
1077,red_noecn,reno,reno,500mbps,4.0,1.0000,415.02,7941.4,0.9999,467.34,5319.8,500000000
1078,red_noecn,reno,reno,500mbps,8.0,1.0000,415.26,7891.0,0.9997,467.66,5286.0,500000000


In [12]:
agg_results_join["RR"] = agg_results_join["total_mean_retx_packets"]/agg_results_join["total_mean_retx_packets_cubic"]
agg_results_join["J_Index"] = agg_results_join["fairness_index"]/agg_results_join["fairness_index_cubic"]
agg_results_join["phi"] = agg_results_join["total_mean_goodput"]/(agg_results_join["speed_bits"]/((1000**2)*8))

In [13]:
agg_results_join

,aqm,cca1,cca2,speed,bdp,fairness_index,total_mean_goodput,total_mean_retx_packets,fairness_index_cubic,total_mean_goodput_cubic,total_mean_retx_packets_cubic,speed_bits,RR,J_Index,phi
0,fifo,bbr,bbr,100mbps,0.5,0.9993,101.64,38771.8,0.9997,100.78,2587.0,100000000,14.987167,0.999600,8.13120
1,fifo,bbr,bbr,100mbps,1.0,0.8975,101.62,40820.6,0.9942,100.84,2398.6,100000000,17.018511,0.902736,8.12960
2,fifo,bbr,bbr,100mbps,2.0,0.9996,101.66,15952.6,0.9985,100.94,2354.6,100000000,6.775079,1.001102,8.13280
3,fifo,bbr,bbr,100mbps,4.0,1.0000,102.08,4192.2,0.9973,102.20,2408.6,100000000,1.740513,1.002707,8.16640
4,fifo,bbr,bbr,100mbps,8.0,0.9999,102.36,4839.4,0.9884,101.56,2790.0,100000000,1.734552,1.011635,8.18880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,red_noecn,reno,reno,500mbps,1.0,1.0000,414.76,7854.0,1.0000,466.90,5325.4,500000000,1.474819,1.000000,6.63616
1076,red_noecn,reno,reno,500mbps,2.0,1.0000,414.94,7915.4,1.0000,467.78,5283.6,500000000,1.498107,1.000000,6.63904
1077,red_noecn,reno,reno,500mbps,4.0,1.0000,415.02,7941.4,0.9999,467.34,5319.8,500000000,1.492800,1.000100,6.64032
1078,red_noecn,reno,reno,500mbps,8.0,1.0000,415.26,7891.0,0.9997,467.66,5286.0,500000000,1.492811,1.000300,6.64416


In [14]:
mean_overal_results = agg_results_join.drop(columns=["bdp", "speed_bits", "fairness_index", "total_mean_goodput", "total_mean_retx_packets", 
                                                "fairness_index_cubic", "total_mean_goodput_cubic", 
                                                "total_mean_retx_packets_cubic", "speed_bits"]).groupby(by=["aqm", "cca1", "cca2", "speed"]).aggregate("mean").reset_index()

mean_overal_results = mean_overal_results.drop(columns=["speed"]).groupby(by=["aqm", "cca1", "cca2"]).aggregate("mean").round(3)

In [15]:
mean_overal_results

RR  J_Index    phi
aqm       cca1  cca2                         
fifo      bbr   bbr    23.164    0.998  7.884
                cubic  14.916    0.806  7.980
          bbr2  bbr2    1.141    0.983  7.960
                cubic   1.823    0.937  7.987
          cubic cubic   1.000    1.000  7.964
          htcp  cubic   1.624    0.974  7.976
                htcp    2.493    1.003  7.994
          reno  cubic   1.010    0.849  7.986
                reno    1.235    0.998  7.976
fq_codel  bbr   bbr    24.468    1.000  7.771
                cubic  13.986    0.994  7.762
          bbr2  bbr2    4.386    1.000  7.818
                cubic   2.312    0.998  7.801
          cubic cubic   1.000    1.000  7.790
          htcp  cubic   1.057    1.000  7.773
                htcp    1.135    1.000  7.751
          reno  cubic   0.891    0.998  7.684
                reno    0.852    1.000  7.520
red       bbr   bbr    38.797    0.955  7.632
                cubic  35.240    0.524  7.624
          bbr2  bbr2    3.943    0.999  7.267
                cubic   3.062    0.715  7.244
          cubic cubic   1.000    1.000  6.507
          htcp  cubic   1.368    0.969  6.526
                htcp    1.614    1.000  6.558
          reno  cubic   1.102    1.000  6.452
                reno    1.212    1.000  6.329
red_noecn bbr   bbr    47.687    0.938  7.503
                cubic  41.056    0.522  7.522
          bbr2  bbr2    4.872    1.000  7.227
                cubic   3.675    0.722  7.211
          cubic cubic   1.000    1.000  6.304
          htcp  cubic   1.272    0.980  6.370
                htcp    1.497    0.999  6.354
          reno  cubic   1.136    1.000  6.129
                reno    1.281    1.000  5.906

In [16]:
mean_overal_results.to_csv("mean_overal_results.csv")